# Fastest Way to Load Data Into PostgreSQL Using Python

https://medium.com/@hakibenita/fastest-way-to-load-data-into-postgresql-using-python-d2e6de8b2aaa

In [1]:
# Converter texto em um objeto datetime.date
import datetime

def parse_first_brewed(text: str) -> datetime.date:
    parts = text.split('/')    
    if len(parts) == 2:
        return datetime.date(int(parts[1]), int(parts[0]), 1)
    elif len(parts) == 1:
        return datetime.date(int(parts[0]), 1, 1)
    else:
        assert False, ('Formato de data desconhecio', text)
    
    
        


In [2]:
parse_first_brewed('09/2007')

datetime.date(2007, 9, 1)

In [3]:
parse_first_brewed('2006')

datetime.date(2006, 1, 1)

In [4]:
# Buscando Dados

from typing import Iterator, Dict, Any
from urllib.parse import urlencode
import requests

def iter_beers_from_api(page_size: int = 5) -> Iterator[Dict[str, Any]]:
    session = requests.Session()
    page = 1
    
    while True:
        response = session.get('https://api.punkapi.com/v2/beers?' + urlencode({
            'page': page,
            'per_page': page_size
        }))
        response.raise_for_status()
        data = response.json()
        if not data:
            break
            
        yield from data # AINDA NÂO SAQUEI ESSE TAL DE YIELD
        
        page += 1
        

In [5]:
beers = iter_beers_from_api()
next(beers)

{'id': 1,
 'name': 'Buzz',
 'tagline': 'A Real Bitter Experience.',
 'first_brewed': '09/2007',
 'description': 'A light, crisp and bitter IPA brewed with English and American hops. A small batch brewed only once.',
 'image_url': 'https://images.punkapi.com/v2/keg.png',
 'abv': 4.5,
 'ibu': 60,
 'target_fg': 1010,
 'target_og': 1044,
 'ebc': 20,
 'srm': 10,
 'ph': 4.4,
 'attenuation_level': 75,
 'volume': {'value': 20, 'unit': 'litres'},
 'boil_volume': {'value': 25, 'unit': 'litres'},
 'method': {'mash_temp': [{'temp': {'value': 64, 'unit': 'celsius'},
    'duration': 75}],
  'fermentation': {'temp': {'value': 19, 'unit': 'celsius'}},
  'twist': None},
 'ingredients': {'malt': [{'name': 'Maris Otter Extra Pale',
    'amount': {'value': 3.3, 'unit': 'kilograms'}},
   {'name': 'Caramalt', 'amount': {'value': 0.2, 'unit': 'kilograms'}},
   {'name': 'Munich', 'amount': {'value': 0.4, 'unit': 'kilograms'}}],
  'hops': [{'name': 'Fuggles',
    'amount': {'value': 25, 'unit': 'grams'},
    '

In [6]:
import psycopg2
from decouple import config



def connect():
    connection = psycopg2.connect(
        host=config('HOST_AWS'),
        database="testload",
        user=config('USER_AWS'),
        password=config('PASSWORD_AWS')
    )
    connection.autocommit = True
    
    return connection

In [7]:
def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS staging_beers;
        CREATE UNLOGGED TABLE staging_beers(
          id                  INTEGER,
            name                TEXT,
            tagline             TEXT,
            first_brewed        DATE,
            description         TEXT,
            image_url           TEXT,
            abv                 DECIMAL,
            ibu                 DECIMAL,
            target_fg           DECIMAL,
            target_og           DECIMAL,
            ebc                 DECIMAL,
            srm                 DECIMAL,
            ph                  DECIMAL,
            attenuation_level   DECIMAL,
            brewers_tips        TEXT,
            contributed_by      TEXT,
            volume              INTEGER
        );
    """)
   

""" If specified, the table is created as an unlogged table. Data written to unlogged tables is 
    not written to the write-ahead log (see Chapter 30), which makes them considerably faster 
    than ordinary tables. However, they are not crash-safe: an unlogged table is automatically 
    truncated after a crash or unclean shutdown. The contents of an unlogged table are also not 
    replicated to standby servers. Any indexes created on an unlogged table are automatically 
    unlogged as well.   
    
    https://www.postgresql.org/docs/current/sql-createtable.html?source=post_page---------------------------#id-1.9.3.85.6
"""

    

' If specified, the table is created as an unlogged table. Data written to unlogged tables is \n    not written to the write-ahead log (see Chapter 30), which makes them considerably faster \n    than ordinary tables. However, they are not crash-safe: an unlogged table is automatically \n    truncated after a crash or unclean shutdown. The contents of an unlogged table are also not \n    replicated to standby servers. Any indexes created on an unlogged table are automatically \n    unlogged as well.   \n    \n    https://www.postgresql.org/docs/current/sql-createtable.html?source=post_page---------------------------#id-1.9.3.85.6\n'

In [8]:
connection = connect()
with connection.cursor() as cursor:
    create_staging_table(cursor)

# Mensurando tempo

### Para mensurar o tempo de cada metodo, vamos criar um modulo(decorator) time

In [9]:
import time
start = time.perf_counter()
time.sleep(1)
elapsed = time.perf_counter() - start
print(f'Time {elapsed:0.4}')

Time 1.001


### a função perf_counter oferece uma maior resolução disponível 

In [51]:
import time
from functools import wraps
from memory_profiler import memory_usage

def profile(fn):
    @wraps(fn)
    def inner(*args, **kwargs):
        fn_kwargs_str = ', '.join(f'{k}={v}' for k, v in kwargs.items())
        print(f'\n{fn.__name__}({fn_kwargs_str})')

        # Measure time
        t = time.perf_counter()
        retval = fn(*args, **kwargs)
        elapsed = time.perf_counter() - t
        print(f'Time   {elapsed:0.4}')

        # Measure memory
        mem, retval = memory_usage((fn, args, kwargs), retval=True, timeout=200, interval=1e-7)

        print(f'Memory {max(mem) - min(mem)}')
        return retval

    return inner

        
        
        
        

In [11]:
@profile
def work(n):
    for i in range(n):
        2 ** n        

In [12]:
work(10)


work()
Time 7.54e-06


In [38]:
work(n=1000)


work(n=1000)
Time 0.0009342


# Benchmark

### No momento que faço esses testes a API retorna 325 cervejas, vamos dá um buster aí né ?!?!?!

In [60]:
beers = list(iter_beers_from_api()) * 100
len(beers)

32500

In [15]:
for beer in beers:
    print(beer['first_brewed'])

09/2007
04/2008
11/2015
09/2013
02/2011
05/2013
07/2012
03/2013
03/2010
01/2011
04/2013
12/2015
11/2009
02/2010
01/2012
01/2012
04/2011
08/2014
06/2013
03/2011
04/2013
12/2008
12/2007
06/2011
10/2008
02/2010
05/2013
02/2012
04/2013
02/2013
08/2009
12/2010
01/2011
01/2014
11/2015
09/2013
10/2008
09/2013
02/2014
01/2014
10/2014
12/2009
07/2014
06/2013
04/2007
03/2012
03/2013
02/2013
02/2013
11/2011
04/2010
04/2007
09/2013
11/2015
01/2013
11/2012
04/2010
11/2008
03/2015
07/2013
06/2014
11/2011
09/2011
06/2014
03/2013
06/2009
05/2012
08/2007
02/2014
12/2011
08/2014
03/2015
12/2014
10/2009
06/2012
05/2013
11/2013
10/2011
08/2011
08/2010
11/2015
01/2016
02/2014
08/2015
10/2010
07/2008
03/2014
09/2013
01/2011
05/2015
03/2012
11/2010
07/2015
06/2013
04/2007
12/2013
03/2015
08/2015
05/2014
12/2015
04/2007
06/2015
12/2013
09/2014
08/2012
10/2010
02/2014
09/2013
11/2012
03/2015
02/2013
05/2014
12/2010
07/2014
03/2015
08/2015
04/2007
02/2015
12/2008
10/2011
07/2009
10/2012
08/2015
10/2008
06/2011


2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
09/2007
04/2008
11/2015
09/2013
02/2011
05/2013
07/2012
03/2013
03/2010
01/2011
04/2013
12/2015
11/2009
02/2010
01/2012
01/2012
04/2011
08/2014
06/2013
03/2011
04/2013
12/2008
12/2007
06/2011
10/2008
02/2010
05/2013
02/2012
04/2013
02/2013
08/2009
12/2010
01/2011
01/2014
11/2015
09/2013
10/2008
09/2013
02/2014
01/2014
10/2014
12/2009
07/2014
06/2013
04/2007
03/2012
03/2013
02/2013
02/2013
11/2011
04/2010
04/2007
09/2013
11/2015
01/2013
11/2012
04/2010
11/2008
03/2015
07/2013
06/2014
11/2011
09/2011
06/2014
03/2013
06/2009
05/2012
08/2007
02/2014
12/2011
08/2014
03/2015
12/2014
10/2009
06/2012
05/2013
11/2013
10/2011
08/2011
08/2010
11/2015
01/2016
02/2014
08/2015
10/2010
07/2008
03/2014
09/2013
01/2011
05/2015
03/2012
11/2010
07/2015
06/2013
04/2007
12/2013
03/2015
08/2015
0

01/2012
04/2009
09/2015
10/2013
11/2014
11/2012
12/2008
09/2009
03/2014
01/2013
06/2010
08/2012
09/2013
11/2013
04/2007
07/2013
06/2010
07/2015
12/2012
11/2013
02/2010
09/2010
08/2014
02/2013
05/2014
02/2012
01/2016
12/2011
06/2013
08/2012
02/2015
02/2015
09/2013
11/2009
11/2014
02/2013
04/2008
11/2013
06/2016
06/2016
03/2016
03/2016
01/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
01/2016
01/2016
01/2016
01/2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2017
2017
2017
2017
2017
2017
2005
2016
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
09/2007
04/2008
11/2015
09/2013
02/2011
05/2013
07/2012
03/2013
03/2010
01/2011
04/2013
1

01/2013
06/2010
08/2012
09/2013
11/2013
04/2007
07/2013
06/2010
07/2015
12/2012
11/2013
02/2010
09/2010
08/2014
02/2013
05/2014
02/2012
01/2016
12/2011
06/2013
08/2012
02/2015
02/2015
09/2013
11/2009
11/2014
02/2013
04/2008
11/2013
06/2016
06/2016
03/2016
03/2016
01/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
01/2016
01/2016
01/2016
01/2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2017
2017
2017
2017
2017
2017
2005
2016
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
09/2007
04/2008
11/2015
09/2013
02/2011
05/2013
07/2012
03/2013
03/2010
01/2011
04/2013
12/2015
11/2009
02/2010
01/2012
01/2012
04/2011
08/2014
06/2013
03/2011
0

06/2011
01/2016
02/2012
07/2012
05/2015
03/2015
02/2012
06/2009
11/2013
09/2011
03/2011
05/2014
12/2009
05/2012
03/2015
03/2015
08/2014
08/2015
07/2015
06/2009
03/2015
01/2011
03/2015
05/2010
09/2008
01/2012
10/2015
04/2015
03/2015
10/2015
10/2011
10/2012
12/2007
03/2014
12/2010
02/2015
06/2009
12/2008
08/2014
03/2015
06/2014
11/2014
03/2012
06/2012
06/2012
08/2009
02/2012
01/2016
03/2015
10/2014
07/2012
10/2011
05/2015
01/2012
04/2009
09/2015
10/2013
11/2014
11/2012
12/2008
09/2009
03/2014
01/2013
06/2010
08/2012
09/2013
11/2013
04/2007
07/2013
06/2010
07/2015
12/2012
11/2013
02/2010
09/2010
08/2014
02/2013
05/2014
02/2012
01/2016
12/2011
06/2013
08/2012
02/2015
02/2015
09/2013
11/2009
11/2014
02/2013
04/2008
11/2013
06/2016
06/2016
03/2016
03/2016
01/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
01/2016
01/2016
01/2016
01/2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2017
2017
2017
2017
2017
2017
2005


...

11/2015
09/2013
02/2011
05/2013
07/2012
03/2013
03/2010
01/2011
04/2013
12/2015
11/2009
02/2010
01/2012
01/2012
04/2011
08/2014
06/2013
03/2011
04/2013
12/2008
12/2007
06/2011
10/2008
02/2010
05/2013
02/2012
04/2013
02/2013
08/2009
12/2010
01/2011
01/2014
11/2015
09/2013
10/2008
09/2013
02/2014
01/2014
10/2014
12/2009
07/2014
06/2013
04/2007
03/2012
03/2013
02/2013
02/2013
11/2011
04/2010
04/2007
09/2013
11/2015
01/2013
11/2012
04/2010
11/2008
03/2015
07/2013
06/2014
11/2011
09/2011
06/2014
03/2013
06/2009
05/2012
08/2007
02/2014
12/2011
08/2014
03/2015
12/2014
10/2009
06/2012
05/2013
11/2013
10/2011
08/2011
08/2010
11/2015
01/2016
02/2014
08/2015
10/2010
07/2008
03/2014
09/2013
01/2011
05/2015
03/2012
11/2010
07/2015
06/2013
04/2007
12/2013
03/2015
08/2015
05/2014
12/2015
04/2007
06/2015
12/2013
09/2014
08/2012
10/2010
02/2014
09/2013
11/2012
03/2015
02/2013
05/2014
12/2010
07/2014
03/2015
08/2015
04/2007
02/2015
12/2008
10/2011
07/2009
10/2012
08/2015
10/2008
06/2011
01/2016
02/2012


04/2007
03/2012
03/2013
02/2013
02/2013
11/2011
04/2010
04/2007
09/2013
11/2015
01/2013
11/2012
04/2010
11/2008
03/2015
07/2013
06/2014
11/2011
09/2011
06/2014
03/2013
06/2009
05/2012
08/2007
02/2014
12/2011
08/2014
03/2015
12/2014
10/2009
06/2012
05/2013
11/2013
10/2011
08/2011
08/2010
11/2015
01/2016
02/2014
08/2015
10/2010
07/2008
03/2014
09/2013
01/2011
05/2015
03/2012
11/2010
07/2015
06/2013
04/2007
12/2013
03/2015
08/2015
05/2014
12/2015
04/2007
06/2015
12/2013
09/2014
08/2012
10/2010
02/2014
09/2013
11/2012
03/2015
02/2013
05/2014
12/2010
07/2014
03/2015
08/2015
04/2007
02/2015
12/2008
10/2011
07/2009
10/2012
08/2015
10/2008
06/2011
01/2016
02/2012
07/2012
05/2015
03/2015
02/2012
06/2009
11/2013
09/2011
03/2011
05/2014
12/2009
05/2012
03/2015
03/2015
08/2014
08/2015
07/2015
06/2009
03/2015
01/2011
03/2015
05/2010
09/2008
01/2012
10/2015
04/2015
03/2015
10/2015
10/2011
10/2012
12/2007
03/2014
12/2010
02/2015
06/2009
12/2008
08/2014
03/2015
06/2014
11/2014
03/2012
06/2012
06/2012


2017
2017
2017
2017
2005
2016
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
09/2007
04/2008
11/2015
09/2013
02/2011
05/2013
07/2012
03/2013
03/2010
01/2011
04/2013
12/2015
11/2009
02/2010
01/2012
01/2012
04/2011
08/2014
06/2013
03/2011
04/2013
12/2008
12/2007
06/2011
10/2008
02/2010
05/2013
02/2012
04/2013
02/2013
08/2009
12/2010
01/2011
01/2014
11/2015
09/2013
10/2008
09/2013
02/2014
01/2014
10/2014
12/2009
07/2014
06/2013
04/2007
03/2012
03/2013
02/2013
02/2013
11/2011
04/2010
04/2007
09/2013
11/2015
01/2013
11/2012
04/2010
11/2008
03/2015
07/2013
06/2014
11/2011
09/2011
06/2014
03/2013
06/2009
05/2012
08/2007
02/2014
12/2011
08/2014
03/2015
12/2014
10/2009
06/2012
05/2013
11/2013
10/2011
08/2011
08/2010


09/2008
01/2012
10/2015
04/2015
03/2015
10/2015
10/2011
10/2012
12/2007
03/2014
12/2010
02/2015
06/2009
12/2008
08/2014
03/2015
06/2014
11/2014
03/2012
06/2012
06/2012
08/2009
02/2012
01/2016
03/2015
10/2014
07/2012
10/2011
05/2015
01/2012
04/2009
09/2015
10/2013
11/2014
11/2012
12/2008
09/2009
03/2014
01/2013
06/2010
08/2012
09/2013
11/2013
04/2007
07/2013
06/2010
07/2015
12/2012
11/2013
02/2010
09/2010
08/2014
02/2013
05/2014
02/2012
01/2016
12/2011
06/2013
08/2012
02/2015
02/2015
09/2013
11/2009
11/2014
02/2013
04/2008
11/2013
06/2016
06/2016
03/2016
03/2016
01/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
06/2016
01/2016
01/2016
01/2016
01/2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2016
2017
2017
2017
2017
2017
2017
2005
2016
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2017
2018
20

2017
2017
2017
2017
2017
2017
2017
2017
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
2018
09/2007
04/2008
11/2015
09/2013
02/2011
05/2013
07/2012
03/2013
03/2010
01/2011
04/2013
12/2015
11/2009
02/2010
01/2012
01/2012
04/2011
08/2014
06/2013
03/2011
04/2013
12/2008
12/2007
06/2011
10/2008
02/2010
05/2013
02/2012
04/2013
02/2013
08/2009
12/2010
01/2011
01/2014
11/2015
09/2013
10/2008
09/2013
02/2014
01/2014
10/2014
12/2009
07/2014
06/2013
04/2007
03/2012
03/2013
02/2013
02/2013
11/2011
04/2010
04/2007
09/2013
11/2015
01/2013
11/2012
04/2010
11/2008
03/2015
07/2013
06/2014
11/2011
09/2011
06/2014
03/2013
06/2009
05/2012
08/2007
02/2014
12/2011
08/2014
03/2015
12/2014
10/2009
06/2012
05/2013
11/2013
10/2011
08/2011
08/2010
11/2015
01/2016
02/2014
08/2015
10/2010
07/2008
03/2014
09/2013
01/2011
05/2015
03/2012
11/2010
07/2015
06/2013
04/2007
12/2013
03/2015
08/2015
05/2014
12/2015
04/2007
06

# Insert Rows One by One

In [16]:
@profile
def insert_one_by_one(connection, beers: Iterator[Dict[str, Any]]) -> None:
    with connection.cursor() as cursor:
        create_staging_table(cursor)
        for beer in beers:
            cursor.execute("""
                INSERT INTO staging_beers VALUES (
                    %(id)s,
                    %(name)s,
                    %(tagline)s,
                    %(first_brewed)s,
                    %(description)s,
                    %(image_url)s,
                    %(abv)s,
                    %(ibu)s,
                    %(target_fg)s,
                    %(target_og)s,
                    %(ebc)s,
                    %(srm)s,
                    %(ph)s,
                    %(attenuation_level)s,
                    %(brewers_tips)s,
                    %(contributed_by)s,
                    %(volume)s
                );
            """, {
                **beer,
                'first_brewed': parse_first_brewed(beer['first_brewed']),
                'volume': beer['volume']['value'],
            })

In [17]:
# print(connect())
# insert_one_by_one(connect(), beers)
# insert_one_by_one

# for beer in beers:
#     print(beer['volume']['value'])

In [18]:
import psycopg2.extras

@profile
def insert_execute_batch(connection, beers: Iterator[Dict[str, Any]]) -> None:
    with connection.cursor() as cursor:
        create_staging_table(cursor)

        all_beers = [{
            **beer,
            'first_brewed': parse_first_brewed(beer['first_brewed']),
            'volume': beer['volume']['value'],
        } for beer in beers]

        psycopg2.extras.execute_batch(cursor, """
            INSERT INTO staging_beers VALUES (
                %(id)s,
                %(name)s,
                %(tagline)s,
                %(first_brewed)s,
                %(description)s,
                %(image_url)s,
                %(abv)s,
                %(ibu)s,
                %(target_fg)s,
                %(target_og)s,
                %(ebc)s,
                %(srm)s,
                %(ph)s,
                %(attenuation_level)s,
                %(brewers_tips)s,
                %(contributed_by)s,
                %(volume)s
            );
        """, all_beers)

In [27]:
insert_execute_batch(connect(), beers)
# insert_execute_batch()


insert_execute_batch()
Time 149.0


In [39]:
@profile
def insert_execute_batch_iterator(connection, beers: Iterator[Dict[str, Any]]) -> None:
    with connection.cursor() as cursor:
        create_staging_table(cursor)

        iter_beers = ({
            **beer,
            'first_brewed': parse_first_brewed(beer['first_brewed']),
            'volume': beer['volume']['value'],
        } for beer in beers)

        psycopg2.extras.execute_batch(cursor, """
            INSERT INTO staging_beers VALUES (
                %(id)s,
                %(name)s,
                %(tagline)s,
                %(first_brewed)s,
                %(description)s,
                %(image_url)s,
                %(abv)s,
                %(ibu)s,
                %(target_fg)s,
                %(target_og)s,
                %(ebc)s,
                %(srm)s,
                %(ph)s,
                %(attenuation_level)s,
                %(brewers_tips)s,
                %(contributed_by)s,
                %(volume)s
            );
        """, iter_beers)

In [54]:
insert_execute_batch_iterator(connect(), beers)


insert_execute_batch_iterator()
Time   112.5
Memory 0.0


In [52]:
@profile
def insert_execute_batch_iterator(
    connection,
    beers: Iterator[Dict[str, Any]],
    page_size: int = 100,) -> None:
    
    with connection.cursor() as cursor:
        create_staging_table(cursor)

        iter_beers = ({
            **beer,
            'first_brewed': parse_first_brewed(beer['first_brewed']),
            'volume': beer['volume']['value'],
        } for beer in beers)

        psycopg2.extras.execute_batch(cursor, """
            INSERT INTO staging_beers VALUES (
                %(id)s,
                %(name)s,
                %(tagline)s,
                %(first_brewed)s,
                %(description)s,
                %(image_url)s,
                %(abv)s,
                %(ibu)s,
                %(target_fg)s,
                %(target_og)s,
                %(ebc)s,
                %(srm)s,
                %(ph)s,
                %(attenuation_level)s,
                %(brewers_tips)s,
                %(contributed_by)s,
                %(volume)s
            );
        """, iter_beers, page_size=page_size)

In [65]:
conn = connect()
insert_execute_batch_iterator(conn, iter(beers), page_size=1000)




insert_execute_batch_iterator(page_size=1000)
Time   45.31
Memory 0.0078125


In [66]:
from typing import Iterator, Optional
import io

class StringIteratorIO(io.TextIOBase):
    def __init__(self, iter: Iterator[str]):
        self._iter = iter
        self._buff = ''

    def readable(self) -> bool:
        return True

    def _read1(self, n: Optional[int] = None) -> str:
        while not self._buff:
            try:
                self._buff = next(self._iter)
            except StopIteration:
                break
        ret = self._buff[:n]
        self._buff = self._buff[len(ret):]
        return ret

    def read(self, n: Optional[int] = None) -> str:
        line = []
        if n is None or n < 0:
            while True:
                m = self._read1()
                if not m:
                    break
                line.append(m)
        else:
            while n > 0:
                m = self._read1(n)
                if not m:
                    break
                n -= len(m)
                line.append(m)
        return ''.join(line)

In [94]:
@profile
def copy_string_iterator(connection, beers: Iterator[Dict[str, Any]]) -> None:
    with connection.cursor() as cursor:
        create_staging_table(cursor)
        beers_string_iterator = StringIteratorIO((
            '|'.join(map(clean_csv_value, (
                beer['id'],
                beer['name'],
                beer['tagline'],
                parse_first_brewed(beer['first_brewed']).isoformat(),
                beer['description'],
                beer['image_url'],
                beer['abv'],
                beer['ibu'],
                beer['target_fg'],
                beer['target_og'],
                beer['ebc'],
                beer['srm'],
                beer['ph'],
                beer['attenuation_level'],
                beer['brewers_tips'],
                beer['contributed_by'],
                beer['volume']['value'],
            ))) + '\n'
            for beer in beers
        ))
        cursor.copy_from(beers_string_iterator, 'beers', sep='|')

In [95]:
copy_string_iterator(connect(), beers)


copy_string_iterator()


UndefinedTable: relation "beers" does not exist


In [92]:
# @profile
def copy_string_iterator(connection, beers: Iterator[Dict[str, Any]], size: int = 8192) -> None:
    
    print(beers)
    
    with connection.cursor() as cursor:
        create_staging_table(cursor)
        beers_string_iterator = StringIteratorIO((
            '|'.join(map(clean_csv_value, (
                beer['id'],
                beer['name'],
                beer['tagline'],
                parse_first_brewed(beer['first_brewed']).isoformat(),
                beer['description'],
                beer['image_url'],
                beer['abv'],
                beer['ibu'],
                beer['target_fg'],
                beer['target_og'],
                beer['ebc'],
                beer['srm'],
                beer['ph'],
                beer['attenuation_level'],
                beer['brewers_tips'],
                beer['contributed_by'],
                beer['volume']['value'],
            ))) + '\n'
            for beer in beers
        ))        
        print(cursor)
        cursor.copy_from(beers_string_iterator, 'beers', sep='|', size=size)


In [93]:
conn = connect()
copy_string_iterator(conn, iter(beers), size=1024)
# beers[0]

<cursor object at 0x7fc7dfa19900; closed: 0>


UndefinedTable: relation "beers" does not exist
